In [1]:
import sys
sys.path.append("../")

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.ensemble import RandomForestClassifier

In [4]:
import pandas as pd
import os,sys
from pathlib import Path
import json 
import logging
logging.basicConfig(format='%(asctime)s %(levelname)-7s %(message)s',
                    stream=sys.stderr, level=logging.INFO)
mpl_logger = logging.getLogger('matplotlib')
mpl_logger.setLevel(logging.INFO)

from sklearn.semi_supervised import LabelPropagation, LabelSpreading

#General ML 
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, silhouette_score,confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from modules.clustering_helpers import select_labeled_samples

#In-house Module Imports
from config import Configuration 
from datasets import EclipseSampledDataset, VoltaSampledDataset
from utils import *

## Guide

This code will generate different labeled data percentages for the selected CV fold.

In [5]:
user = "aksar"
logging.warning(f'Are you sure that you are: {user}?')

2022-05-05 12:46:57,975 WARNING Are you sure that you are: aksar?


In [28]:
#Update these
CV_INDEX = 4
OUTER_DIR = "active_learning_experiments_final_hdfs"
SYSTEM = 'eclipse'
FE_NAME = 'tsfresh'
EXP_NAME = f'{FE_NAME}_experiments'

In [29]:
OUTPUT_DIR = f'/projectnb/peaclab-mon/{user}/{OUTER_DIR}'
FEATURE_SELECTION = False
SCALER = 'None' #For now, do the scaling inside the notebook, then you can move that to the class function
MODEL_CONFIG = f'{SYSTEM}_labeled_data_gen'
logging.warning('Results will be generated in {}, double check please!'.format(MODEL_CONFIG))

2022-05-05 13:21:32,596 WARNING Results will be generated in eclipse_labeled_data_gen, double check please!


In [30]:
conf = Configuration(ipython=True,
                     overrides={
                         'output_dir': Path(OUTPUT_DIR), #change
                         'system' : SYSTEM,
                         'exp_name':EXP_NAME,                                                  
                         'cv_fold':CV_INDEX, 
                         'model_config': MODEL_CONFIG,
                     })

with open(str(conf['experiment_dir']) + '/anom_dict.json') as f:
    ANOM_DICT = json.load(f)
with open(str(conf['experiment_dir']) + '/app_dict.json') as f:
    APP_DICT = json.load(f) 
    
APP_REVERSE_DICT = {}
for app_name, app_encoding in APP_DICT.items():
    APP_REVERSE_DICT[app_encoding] = app_name    

ANOM_REVERSE_DICT = {}
for anom_name, anom_encoding in ANOM_DICT.items():
    ANOM_REVERSE_DICT[anom_encoding] = anom_name

2022-05-05 13:21:32,629 WARNING You set windowing False, but you are trying to define window_size parameter, it is automatically set to 0. Please double check.
2022-05-05 13:21:32,629 INFO    Setting directory names
2022-05-05 13:21:32,648 INFO    Saving configuration as CSV


# The configuration used for this run:
# {'cv_fold': 4,
#  'exp_name': 'tsfresh_experiments',
#  'experiment_dir': PosixPath('/projectnb/peaclab-mon/aksar/active_learning_experiments_final_hdfs/eclipse/tsfresh_experiments'),
#  'feature_extract': False,
#  'feature_select': False,
#  'hdf_data_path': PosixPath('/projectnb/peaclab-mon/aksar/datasets/eclipse_final_hdfs'),
#  'metadata_path': None,
#  'model_config': 'eclipse_labeled_data_gen',
#  'model_config_dir': PosixPath('/projectnb/peaclab-mon/aksar/active_learning_experiments_final_hdfs/eclipse/tsfresh_experiments/CV_4/eclipse_labeled_data_gen'),
#  'model_dir': PosixPath('/projectnb/peaclab-mon/aksar/active_learning_experiments_final_hdfs/eclipse/tsfresh_experiments/CV_4/eclipse_labeled_data_gen/model'),
#  'num_split': 5,
#  'operation': 'read',
#  'output_dir': PosixPath('/projectnb/peaclab-mon/aksar/active_learning_experiments_final_hdfs/eclipse'),
#  'plots_dir': PosixPath('/projectnb/peaclab-mon/aksar/active_learning_experim

In [31]:
if SYSTEM == 'eclipse':
    eclipseDataset = EclipseSampledDataset(conf)
    train_data, train_label, test_data, test_label = eclipseDataset.load_dataset(
                                                         cv_fold=CV_INDEX, 
                                                         scaler=SCALER,
                                                         borghesi=False, 
                                                         mvts=True if FE_NAME == 'mvts' else False, 
                                                         tsfresh=True if FE_NAME == 'tsfresh' else False)
                                                                                 
elif SYSTEM == 'volta':
    voltaDataset = VoltaSampledDataset(conf)
    train_data, train_label, test_data, test_label = voltaDataset.load_dataset(
                                                           cv_fold=CV_INDEX,        
                                                           scaler=SCALER,
                                                           borghesi=False,
                                                           mvts=True if FE_NAME == 'mvts' else False, 
                                                           tsfresh=True if FE_NAME == 'tsfresh' else False)
                                                                               
                                                                               
assert list(train_data.index) == list(train_label.index) #check the order of the labels     
assert list(test_data.index) == list(test_label.index) #check the order of the labels    

if FEATURE_SELECTION:
    selected_features = pd.read_csv(conf['experiment_dir'] / 'selected_features.csv')
    train_data = train_data[list(selected_features['0'].values)]
    test_data = test_data[list(selected_features['0'].values)]
    
train_label['anom_names'] = train_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)
test_label['anom_names'] = test_label.apply(lambda x: ANOM_REVERSE_DICT[x['anom']], axis=1)

all_data = pd.concat([train_data, test_data])
all_data = all_data.dropna(axis=1, how='any')
all_label = pd.concat([train_label,test_label])

train_data = all_data.loc[train_label.index]
test_data = all_data.loc[test_label.index]
    
logging.info("Train data shape %s",train_data.shape)
logging.info("Train label shape %s",train_label.shape)
logging.info("Test data shape %s",test_data.shape)  
logging.info("Test label shape %s",test_label.shape)

logging.info("Train data label dist: \n%s",train_label['anom'].value_counts())
logging.info("Test data label dist: \n%s",test_label['anom'].value_counts())  

2022-05-05 13:21:32,692 INFO    BaseDataset Class Initialization
2022-05-05 13:21:32,693 INFO    HPCDataset Class Initialization
2022-05-05 13:21:32,693 INFO    EclipseSampledDataset Class Initialization
2022-05-05 13:23:09,262 INFO    Train data shape (5616, 121836)
2022-05-05 13:23:09,263 INFO    Train label shape (5616, 2)
2022-05-05 13:23:09,264 INFO    Test data shape (18950, 121836)
2022-05-05 13:23:09,264 INFO    Test label shape (18950, 2)
2022-05-05 13:23:09,266 WARNING Beware that no scaling method is applied
2022-05-05 13:24:27,017 INFO    Train data shape (5616, 80839)
2022-05-05 13:24:27,019 INFO    Train label shape (5616, 3)
2022-05-05 13:24:27,019 INFO    Test data shape (18950, 80839)
2022-05-05 13:24:27,020 INFO    Test label shape (18950, 3)
2022-05-05 13:24:27,024 INFO    Train data label dist: 
0    5055
4     167
3     166
2     115
1     113
Name: anom, dtype: int64
2022-05-05 13:24:27,027 INFO    Test data label dist: 
4    5240
3    5205
2    3627
1    3608
0  

In [32]:
if SYSTEM == 'eclipse':
    num_samples_per_pair = [1,3,5]
    num_of_app = 6
elif SYSTEM == 'volta':
    num_samples_per_pair = [1,7,13]
    num_of_app = 11
else:
    raise

In [33]:
for num_sample_per_pair in num_samples_per_pair:
    
    labeled_train_label, labeled_test_label, node_indices_labeled  = select_labeled_samples(train_label, num_sample_per_pair, num_of_app)

    logging.info("Labeled data label dist: \n%s",labeled_train_label['anom'].value_counts())
    logging.info("Unlabeled data label dist: \n%s",labeled_test_label['anom'].value_counts()) 

    labeled_train_label.to_csv(conf['experiment_dir'] / f'CV_{CV_INDEX}'/ f'labeled_train_label_{num_sample_per_pair}.csv')
    labeled_test_label.to_csv(conf['experiment_dir'] / f'CV_{CV_INDEX}'/ f'labeled_test_label_{num_sample_per_pair}.csv')

2022-05-05 13:24:27,575 INFO    Labeled data label dist: 
4    8
3    8
2    7
1    7
0    6
Name: anom, dtype: int64
2022-05-05 13:24:27,578 INFO    Unlabeled data label dist: 
0    5049
4     159
3     158
2     108
1     106
Name: anom, dtype: int64
2022-05-05 13:24:27,697 INFO    Labeled data label dist: 
4    27
3    27
2    18
1    18
0    18
Name: anom, dtype: int64
2022-05-05 13:24:27,700 INFO    Unlabeled data label dist: 
0    5037
4     140
3     139
2      97
1      95
Name: anom, dtype: int64
2022-05-05 13:24:27,813 INFO    Labeled data label dist: 
4    44
3    43
2    31
1    31
0    30
Name: anom, dtype: int64
2022-05-05 13:24:27,816 INFO    Unlabeled data label dist: 
0    5025
3     123
4     123
2      84
1      82
Name: anom, dtype: int64
